ALIEXPRESS WEBSCRAPER

In [117]:
#driver = webdriver.Chrome() 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - Downloading: 100%|██████████| 6.81M/6.81M [00:00<00:00, 8.80MB/s]


-----FUNCTION DEFINITION & MODULES IMPORT-----

In [279]:
# IMPORTING MODULES
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import random 
import numpy as np


In [223]:
#THIS FUNCTION APPLY A TIME.SLEEP WITH A RANDOM VALUE WHICH IS CHOSEN BETWEEN 2 INPUTS:
#INPUT1=T1  -> MINIMUM TIME SLEEP
#INPUT2=T12 -> MAXIMUM TIME SLEEP
def time_randomizer (t1,t2):
    ran_time=random.uniform(t1,t2)
    time.sleep(ran_time)

In [255]:
def selenium_init(url):
    driver = webdriver.Chrome()  # Activate driver for Chrome in this case
    time_randomizer(1,3)
    # Open Url in the browser
    driver.get(url)
    time_randomizer(2,4)
    return driver

In [226]:
def enter_aliexpress():
    url='https://es.aliexpress.com/'
    driver=selenium_init(url)
    title = driver.title
    print('Webpage Title:', title)
    if len(title)>0:
        print('Successful connection \n')
    else:
        print('Conection failed \n')
    return driver

In [293]:
def starting_setup ():
    
    #ENTERING ALIEXPRESS
    driver = enter_aliexpress() 
    html=driver.page_source
    soup = BeautifulSoup(html)

    #ACCEPTING COOKIES
    time_randomizer(2,7)
    cookies=driver.find_element(By.CLASS_NAME, "btn-accept")
    cookies.click()
    driver.maximize_window() 

    #GOING TO VIDEOJUEGOS
    time_randomizer(3,7)
    driver.get("https://www.aliexpress.com/category/44/consumer-electronics.html")
    time_randomizer(2,5)
    videojuegos=driver.find_elements(By.XPATH, "//*[contains(text(), 'Videojuegos')]")[1]
    videojuegos.click()

    #SORTING BY NUMBER OF ORDERS
    driver.get("https://es.aliexpress.com/category/204003301/video-games.html")
    time_randomizer(3,6)
    sort_by_orders=driver.find_elements(By.XPATH, "//*[contains(text(),'Pedidos')]")[2]
    sort_by_orders.click()

    #SORTING BY RATING --> RATING>=4
    driver.get("https://es.aliexpress.com/category/204003301/video-games.html?CatId=204003301&g=y&isCategoryBrowse=true&isrefine=y&sortType=total_tranpro_desc&trafficChannel=main")
    time_randomizer(2,5)
    sort_by_rating=driver.find_elements(By.CLASS_NAME, "comet-checkbox-icon")[3]
    sort_by_rating.click()

    #SCROLLING TILL THE BOTTOM OF THE WEBPAGE
    for y in range (1,13):
        height = y*1080
        driver.execute_script(f"window.scrollTo(0, {height})") 
        time_randomizer(1,4)

    return driver


In [330]:
#THIS FUNCTION SCRAP THE DATA FROM THE CURRENT WEBPAGE & RETURNS A DATAFRAME
def page_scraper(html):
    soup=BeautifulSoup(html)
    #GETTING DATA
    product_names=soup.select('h1[class*="manhattan--titleText--"]')
    product_real_price=soup.select('div[class*="manhattan--price-sale--"]')
    product_original_price=soup.select('div[class*="manhattan--price-original--"]')
    product_sell_qty=soup.select('span[class*="manhattan--trade--"]')
    product_rate=soup.select('span[class*="manhattan--evaluation--"]')
    product_shipping_info=soup.select('span[class*="tag--textStyle--"]')
    product_official_store=soup.select('span[class*="cards--store--"]')
    #APPENDING DATA INTO LISTS
    product_names_list=[product.text for product in product_names]
    product_real_price_list=[product.text for product in product_real_price]
    product_original_price_list=[product.text for product in product_original_price]
    product_sell_qty_list=[product.text for product in product_sell_qty]
    product_rate_list=[product.text for product in product_rate]
    product_shipping_info_list=[product.text for product in product_shipping_info]
    product_official_store_list=[product.text for product in product_official_store]
    #CREATING DATAFRAME
    df=pd.DataFrame({'Product Name':product_names_list,
                            'Price':product_real_price_list,
                            'Original Price':product_original_price_list,
                            'Sell Quantity':product_sell_qty_list,
                            'Official Store':product_official_store_list,
                            'Rating':product_rate_list})
    return df

-----MAIN-----

In [349]:
#MAIN
driver = starting_setup()
url=driver.current_url
html=driver.page_source
global_df=pd.DataFrame()
print('WE ARE IN THE FIRST PAGE')
for i in range (2,50):
    driver.get(url)
    time_randomizer(6,13)
    
    for y in range (1,13):
        height = y*1080
        driver.execute_script(f"window.scrollTo(0, {height})") 
        time_randomizer(1,4)
    html=driver.page_source
    df=page_scraper(html)
    global_df=pd.concat([global_df,df],ignore_index=True)
    if "page=" in url:
        url=url.replace('page=/d+',f'page={i}')
        print(f'WE ARE IN THE PAGE NUMBER {i}')
    else:
        url= f'{url}&page=2'
        print(url)
        print(f'WE ARE IN THE PAGE NUMBER {i}')
global_df.to_csv()
driver.quit()

Webpage Title: AliExpress - Compra online de Electrónica, Moda, Casa y jardín, Deportes y ocio, Motor y seguridad, y más. - AliExpress - AliExpress
Successful connection 

ESTAMOS EN LA PRIMERA PAGINA
https://es.aliexpress.com/category/204003301/video-games.html?CatId=204003301&g=y&isCategoryBrowse=true&isrefine=y&sortType=total_tranpro_desc&trafficChannel=main&isFavorite=y&page=2
ESTAMOS EN LA PAGINA2
ESTAMOS EN LA PRIMERA PAGINA
ESTAMOS EN LA PAGINA 3
ESTAMOS EN LA PRIMERA PAGINA
ESTAMOS EN LA PAGINA 4
ESTAMOS EN LA PRIMERA PAGINA
ESTAMOS EN LA PAGINA 5
ESTAMOS EN LA PRIMERA PAGINA
ESTAMOS EN LA PAGINA 6
ESTAMOS EN LA PRIMERA PAGINA
ESTAMOS EN LA PAGINA 7
ESTAMOS EN LA PRIMERA PAGINA
ESTAMOS EN LA PAGINA 8
ESTAMOS EN LA PRIMERA PAGINA
ESTAMOS EN LA PAGINA 9
ESTAMOS EN LA PRIMERA PAGINA
ESTAMOS EN LA PAGINA 10
ESTAMOS EN LA PRIMERA PAGINA
ESTAMOS EN LA PAGINA 11
ESTAMOS EN LA PRIMERA PAGINA
ESTAMOS EN LA PAGINA 12
ESTAMOS EN LA PRIMERA PAGINA
ESTAMOS EN LA PAGINA 13
ESTAMOS EN LA PRIM

In [351]:
global_df.to_csv()

',Product Name,Price,Original Price,Sell Quantity,Official Store,Rating\r\n0,"Consola de videojuegos portátil con más de 14000 juegos, 64GB, mando inalámbrico para PS1, GBA, MAME, GBMD, NEO ","16,03€","55,32€",10.000+ vendido(s),Factory Direct Collected Store,4.5\r\n1,"Mini consola de videojuegos portátil Retro para niños, reproductor de juegos de 8 bits, pantalla LCD a Color de 3,0 pulgadas, 500 juegos integrados ","3,08€","22,42€",10.000+ vendido(s),Factory Direct Collected Store,4.6\r\n2,"Mini consola de videojuegos portátil Retro para niños, reproductor de juegos de 8 bits, pantalla LCD a Color de 3,0 pulgadas, 500 juegos integrados ","6,51€","23,01€",10.000+ vendido(s),Digitaling Store,4.6\r\n3,"Mando a distancia para teléfono móvil VR, gafas, Android inalámbrico, mando para teléfono inteligente, E-book, palanca de giro de página ","1,43€","11,2€",10.000+ vendido(s),Digitaling Store,4.6\r\n4,"8bitdo-controlador de juego inalámbrico Ultimate 2,4G con base de carga para PC, Windows 

In [239]:
'''product_shipping_time=[]
product_free_shipping=[]
product_discount=[]
for text in product_shipping_info_list:
    if 'dto'or'descuento' in text:
        product_discount.append(text)
    else:
        product_discount.append('-')
    if 'Envío' in text:
        product_free_shipping.append(text)
    else:
        product_free_shipping.append('-')
    if 'Entrega' in text:
        product_shipping_time.append(text)
    else:
        product_shipping_time.append('-')

print(len(product_shipping_time))
print(len(product_free_shipping))
print(len(product_discount))
'''

115
115
115
